In [1]:
import numpy as np
import cv2
import os
# import scipy
# import matplotlib.pylab as plt
# from skimage import io

In [2]:
def preprocessing(template) :
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (template, template_gray)

In [3]:
template = cv2.imread('./images/final_exam/Templates/Template-3.png')
template, template_gray = preprocessing(template)
print(template.shape)


(469, 347, 3)


In [4]:
params_dir = os.getcwd()+'/new-camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

c:\Users\gobes\OneDrive\Desktop\Fi\010723305-main\010723305-main/new-camera_params/
Camera matrix
[[1.29968396e+03 0.00000000e+00 6.32814378e+02]
 [0.00000000e+00 1.29933998e+03 4.74721903e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Len distortion
[[-0.40795692  0.24741507  0.00200567 -0.00202735 -0.09445672]]


In [5]:
def crop_image(ref_img, query_img, H) : # Warp and crop
    rows, cols = ref_img.shape[:2]
    warped_img = cv2.warpPerspective(query_img, H, (2*cols, 2*rows))
    warped_img[0:rows, 0:cols] = ref_img
    grayscale = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    mask = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)[1]
    # find contour inside mask 
    contour, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # get contour with maximum area
    max_area_contour = max(contour, key=cv2.contourArea)

    # create bounding box
    (x, y, w, h) = cv2.boundingRect(max_area_contour)

    result = warped_img[:h, :w]
    return result

In [6]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [7]:
def H_from_ransac(fp, tp, model, maxiter=1000, match_threshold = 10) :
    """Homography matrix from RANSAC robust estimator"""
    data = np.vstack((fp,tp))
    H, ransac_data = ransac(data.T, model, 4, maxiter, match_threshold, 10, return_all=True)
    return H, ransac_data['inliers']

In [8]:

cap = cv2.VideoCapture('./videos/final_exam/Dataset-2/left_output.avi')

while cap.isOpened() :
    ret, frame = cap.read()
    # vdo_fram = frame.copy()
    if ret :
       cv2.imshow('frame',frame)
        frame, frame_gray = preprocessing(frame)

        # sift = cv2.SIFT_create()
        # bf = cv2.BFMatcher()
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        frame_kpts, frame_desc = sift.detectAndCompute(frame_gray, None)

        matches = bf.knnMatch(template_desc, frame_desc, k=2)
        good_matches = list()
        good_matches_list = list()

        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])

        drawmatch_img = cv2.drawMatchesKnn(template, template_kpts, frame, frame_kpts, good_matches_list, None, flags=2)

        MIN_MATCH_NUMBER = 10
        
        if len(good_matches) > MIN_MATCH_NUMBER :

            tp = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            fp = np.float32([ frame_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            
            H, inlier_masks = cv2.findHomography(fp, tp, cv2.RANSAC, 10.0)
            
            ransac_img = cv2.drawMatchesKnn(template, template_kpts, frame, frame_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            h, w = template.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template, template_kpts, frame, frame_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
            
            cv2.imshow('detected frame', detected_img)
        else :
            
            cv2.imshow('detected frame', frame)
        key = cv2.waitKey(33) & 0xFF
        
        if key == ord('q') : # this line control the period between image frame
            break
        else :
           

        cap.release()
        cv2.destroyAllWindows()

IndentationError: unexpected indent (Temp/ipykernel_28324/1158761826.py, line 8)

In [4]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.6*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return

In [8]:
cap = cv2.VideoCapture('./videos/final_exam/Dataset-2/left_output.avi')

while cap.isOpened() :
    #Read new frame
    ret, frame = cap.read()
    if ret == True :

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected, drawmatch =  feature_object_detection(template, template_gray, frame, frame_gray, 10)
        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(33) & 0xFF == ord('q') : # Period control f - 1/T
            break
    else :
        break
    
cap.release()
cv2.destroyAllWindows()

Keypoints not enough


TypeError: cannot unpack non-iterable NoneType object